<hr style="border-width:2px;border-color:##75DFC1">
<center> <h1> Project </h1> </center> <br>
<center> <h2> Movie recommendation system  </h2> </center> <br>
<hr style="border-width:2px;border-color:##75DFC1">



*Description of the Project:*

Creation of a recommendation system:

*   collaborative filtering application
*   consolidation of a dataset by web scraping
*   application of Deep Learning algorithms.

Data Resources:


*   Notes available here: https://grouplens.org/datasets/movielens/20m/

* Movie info available here: https://www.imdb.com/interfaces/

* Possibility to scrape imdb, senscritique, rottentomatoes, …

Project Members: (alphabetically w.r.t. surnames)

*  Femke Lutz
*  Athoy Nilima
*  Clemens Schaab
*  Elvid Srebric






**Step 1: Data mining- Movielens**


*Data Files in this google Drive:*

*Movielens data*

*   Data/ml-20m/genome-scores.csv
*   Data/ml-20m/genome-tags.csv
*   Data/ml-20m/links.csv
*   Data/ml-20m/movies.csv
*   Data/ml-20m/ratings.csv
*   Data/ml-20m/tags.csv



# Preliminary tasks

In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import os
%pwd

'/Users/Shared/Work/BootCamp/Project/MovieRecommendation/Notebooks'

In [2]:
# For Google Collab only:
# Define the path to file
# First create a shortcut to the folder created by Elvid to your drive
# Then mount google drive with the following set of commands

#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/DS_Bootcamp_Movie_Recommendation_System

In [3]:
# Read Data Files into pandas dataframes
df_genome_scores=pd.read_csv("../Data/ml-20m/genome-scores.csv")
df_genome_tags=pd.read_csv("../Data/ml-20m/genome-tags.csv")
df_links=pd.read_csv("../Data/ml-20m/links.csv")
df_movies=pd.read_csv("../Data/ml-20m/movies.csv")

#Movie Data

##Check Data: Tag Genome

***Tag Genome***

[genome-paper]: http://files.grouplens.org/papers/tag_genome.pdf

The tag genome is a data structure that contains tag relevance scores for movies.  The structure is a dense matrix: each movie in the genome has a value for *every* tag in the genome.

As described in [this article][genome-paper], the tag genome encodes how strongly movies exhibit particular properties represented by tags (atmospheric, thought-provoking, realistic, etc.). The tag genome was computed using a machine learning algorithm on user-contributed content including tags, ratings, and textual reviews.


### Genome Scores

The file `genome-scores.csv` contains movie-tag relevance data in the following format:

    movieId,tagId,relevance


In [4]:
# Check the data- head method
df_genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [5]:
# Check the data- info
df_genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


###Genome tags
`genome-tags.csv`, provides the tag descriptions for the tag IDs in the genome file, in the following format:

    tagId,tag

The `tagId` values are generated when the data set is exported, so they may vary from version to version of the MovieLens data sets.

In [6]:
# Check the data- head method
df_genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [7]:
# Check the data- info
df_genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


###Merging these two data sets for a complete Genome data file

In [8]:
# Merge dataframes
df_genome=df_genome_scores.merge(df_genome_tags,on="tagId",how="outer")

df_genome.head()

,movieId,tagId,relevance,tag
0,1,1,0.02500,007
1,2,1,0.03975,007
2,3,1,0.04350,007
3,4,1,0.03725,007
4,5,1,0.04200,007


In [9]:
df_genome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11709768 entries, 0 to 11709767
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
 3   tag        object 
dtypes: float64(1), int64(2), object(1)
memory usage: 446.7+ MB


In [10]:
df_genome.columns

Index(['movieId', 'tagId', 'relevance', 'tag'], dtype='object')

##Check Data:  Movies


In [11]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [13]:
df_movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')


##Check Data: Links

In [14]:
# Check the data- head method
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [15]:
#info
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [16]:
#columns
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


** Merging this file with previous ones would be based on the 
movieID column**

##Merge

In [17]:
# We have three dataframes so far: df_genome, df_movies, df_links
# These dataframes have one column common: movieId
# independent of individual users

lists_movieData=[df_genome, df_movies, df_links]
import functools as ft
movieData= ft.reduce(lambda left, right: pd.merge(left, right, on='movieId'), lists_movieData)

movieData.head()

,movieId,tagId,relevance,tag,title,genres,imdbId,tmdbId
0,1,1,0.02500,007,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,1,2,0.02500,007 (series),Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
2,1,3,0.05775,18th century,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
3,1,4,0.09675,1920s,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
4,1,5,0.14675,1930s,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0


##Save data in a separate file for future correspondence


In [18]:
%mkdir -p "../CombinedData"
movieData.to_csv("../CombinedData/movieData_ml_20m.csv")

In [19]:
movieData.columns

Index(['movieId', 'tagId', 'relevance', 'tag', 'title', 'genres', 'imdbId',
       'tmdbId'],
      dtype='object')